# Run Watson Classifiers in Real-time Configuration

###### Author: Alberto Nieto
###### Organization: Esri
###### Date: June, 2017

# Pseudocode

Goal: To run validation routines on the classification pipeline. 

Procedure:
1. Authenticate
2. Retrieve Models
3. Prepare Unittest data - Images with known Labels
4. Classify and record output results
5. Build confusion matrix 
6. Create classification diagnostics

# 1. Routine Setup

Imported modules

In [ ]:
import os
from watson_developer_cloud import VisualRecognitionV3
import arcgis
import pandas as pd
from copy import deepcopy
import requests
from bs4 import BeautifulSoup
import json
from IPython.display import Image, display
from IPython.core.display import HTML 
import scipy.misc
from datetime import datetime, timedelta
import time

workspace

In [ ]:
workspace_dir = os.getcwd()+"\\Temp"

## IBM Watson Authentication

In [ ]:
visual_recognition = VisualRecognitionV3(
    '2018-03-19',
    iam_api_key=os.environ['BLUEMIX_API_KEY'])

## 1b. Retrieve Models

In [ ]:
models_dict = visual_recognition.list_classifiers(verbose=True); models_dict

In [ ]:
models_df = pd.DataFrame.from_dict(models_dict['classifiers']); models_df

In [ ]:
model_128_name = 'Esri_Classification_CobbCounty_Cam128'
model_129_name = 'Esri_Classification_CobbCounty_Cam129'
model_130_name = 'Esri_Classification_CobbCounty_Cam130'

In [ ]:
model_128_id = models_df.loc[models_df['name'] == model_128_name, 'classifier_id'].values[0]
model_129_id = models_df.loc[models_df['name'] == model_129_name, 'classifier_id'].values[0]
model_130_id = models_df.loc[models_df['name'] == model_130_name, 'classifier_id'].values[0]

In [ ]:
cameraid_model_dict = {
    "128": model_128_id,
    "129": model_129_id,
    "130": model_130_id
}

## 1c. Retrieve input images location

In [ ]:
images_url = r"https://www.cobbgis.org/openimage/bravescam"
cams = ["128", "129", "130"]
cam_urls = ["{0}/Cam{1}".format(images_url, cam) for cam in cams]; cam_urls

# 2. Set Helper Functions

## 2b. Classify Image

In [ ]:
def classify_image(image, classifier_id, threshold_val='0.15', display_image=False):
    """
    Pass a locally stored image to a Watson classifier and return the classification results
    :param image: String - Full path to locally stored image
    :param classifier_id: String - Classifier ID
    :param threshold_val: String - Numeric value in string format that establishes which classes are returned
    :return: JSON - classes determined by Watson classifier
    """ 
    if display_image:
        display(Image(image))
    with open(image, 'rb') as images_file:
        classes = visual_recognition.classify(
            images_file,
            threshold=threshold_val,
            classifier_ids=classifier_id)
        output_class = classes['images'][0]['classifiers'][0]['classes'][0]['class']
    return output_class

In [ ]:
# Test function
test_image, test_image_url = get_latest_snapshot(cam_urls[0], workspace_dir)
Image(test_image)
test_label = classify_image(test_image, model_128_id, display_image=True)
test_label

# 2e. Iteration Helper (i.e. main)

In [ ]:
def classify_and_for_time_period(camera_urls_list, time_duration, time_stride,
                                 images_webpage_url, workspace,
                                 camera_to_model_dict,
                                 target_service, target_label_field, target_camera_id_field, target_source_image_field,    
                                 threshold_val='0.15', display_image=False, overwrite_attachment=True, layer_index=0, 
                                 edit="update", verbose=False):
    
    end_time = datetime.now() + timedelta(seconds=time_duration)
    while datetime.now() < end_time:
        print("\n>>>>Performing Classification sweep at {0}<<<<".format(datetime.now()))
        
        # Iterate on each camera
        for cam_url in camera_urls_list:
            camera_id = cam_url[-3:]
            
            print("\nClassifying current pedestrian activity for camera {0}...".format(str(camera_id)))
            classifier_id = cameraid_model_dict[camera_id]
            if verbose:
                print("\tApplying model {0}...".format(classifier_id))
            if verbose:
                print("\tRetrieving snapshot...")
            image, image_url = get_latest_snapshot(cam_url, workspace)
            if verbose:
                print("\tClassifying snapshot...")
            label = classify_image(image, classifier_id, display_image=False)

            if edit == "update":
                if verbose:
                    print("\tUpdating feature...")
                update_hosted_service_feature(label, camera_id, image_url, image, object_point_srvc, "pedestrian_density", "camera_id", "source_image")
            elif edit == "add":
                if verbose:
                    print("\tAdding feature...")
                add_hosted_service_feature(label, camera_id, image_url, image, object_point_srvc, "pedestrian_density", "camera_id", "source_image")
                
        # Add time_stride pause
        time.sleep(time_stride)

In [ ]:
# Test main function
camera_urls_list = cam_urls
time_duration = 60
time_stride = 10

images_webpage_url = images_url
workspace = workspace_dir
camera_to_model_dict = cameraid_model_dict

target_service = object_point_srvc
target_label_field = label_attribute_name
target_camera_id_field = camera_id_attribute_name
target_source_image_field = source_image_attribute_name

classify_and_for_time_period(camera_urls_list, time_duration, time_stride,
                             images_webpage_url, workspace,
                             camera_to_model_dict,
                             target_service, target_label_field, target_camera_id_field, target_source_image_field,    
                             threshold_val='0.15', display_image=False, overwrite_attachment=True, layer_index=0, 
                             edit="update", verbose=True)

# 3. Run Validation

In [ ]:
camera_urls_list = cam_urls
time_stride = 10

images_webpage_url = images_url
workspace = workspace_dir
camera_to_model_dict = cameraid_model_dict

target_service = object_point_srvc
target_label_field = label_attribute_name
target_camera_id_field = camera_id_attribute_name
target_source_image_field = source_image_attribute_name

threshold_val = '0.15'
display_image = False
overwrite_attachment = True
layer_index = 0
edit = "add"
verbose = True

classify_and_for_time_period(camera_urls_list, time_duration, time_stride,
                             images_webpage_url, workspace,
                             camera_to_model_dict,
                             target_service, target_label_field, target_camera_id_field, target_source_image_field,    
                             threshold_val=threshold_val, display_image=display_image, 
                             overwrite_attachment=overwrite_attachment, layer_index=layer_index, 
                             edit=edit, verbose=verbose)